Segmenting and Clustering Neighborhoods in Toronto:

Before we get the data and start exploring it, let's download all the dependencies that we will need

In [6]:
import numpy as np #vectorized manner

import pandas as pd #data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json #JSON files


from geopy.geocoders import Nominatim #latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

from bs4 import BeautifulSoup


print('Libraries imported.')

Libraries imported.


In [1]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed and imported!')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-4.0.0               |             py_0         606 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be 

Getting page data from Wikipedia

In [7]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')


Extracting table data from Wikipedia with unique values

In [49]:
table_can_zipinfo = soup.find('table')
colvals = table_can_zipinfo.find_all('td')

elem_cnt = len(colvals)

postcode = []
borough = []
neighborhood = []

for i in range(0, elem_cnt, 3):
    postcode.append(colvals[i].text.strip())
    borough.append(colvals[i+1].text.strip())
    neighborhood.append(colvals[i+2].text.strip())

Pandas Dataframe

In [50]:
df_toronto = pd.DataFrame(data=[postcode, borough, neighborhood]).transpose()
df_toronto.columns = ['Postcode', 'Borough', 'Neighborhood']

Pandas Dataframe: Cleanse the data

In [57]:
df_toronto.drop(df_toronto[df_toronto['Borough'] == 'Not assigned'].index, inplace=True)
df_toronto.loc[df_toronto.Neighborhood == 'Not assigned', "Neighborhood"] = df_toronto.Borough

In [ ]:
Group the data: Postcode & Borough

In [58]:
df_toronto = df_toronto.groupby(['Postcode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df_toronto.columns = ['Postcode', 'Borough', 'Neighborhood']

In [59]:
df_toronto.shape

(103, 3)

NameError: name 'neighborhoods' is not defined

Read the Geospatial csv file and inner join it

In [12]:
df_latlng = pd.read_csv('http://cocl.us/Geospatial_data')
df_latlng.columns = ['Postcode', 'Latitude', 'Longitude']

In [13]:
df_join = pd.merge(df_toronto, df_latlng, on=['Postcode'], how='inner')

In [14]:
df_join.head(5)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M6M,York,"Keelesdale, Mount Dennis, Silverthorn",43.691116,-79.476013
1,M1G,Scarborough,Woburn,43.770992,-79.216917
2,M4E,East Toronto,The Beaches,43.676357,-79.293031
3,M4C,East York,Woodbine Heights,43.695344,-79.318389
4,M6H,West Toronto,Dovercourt Village,43.669005,-79.442259


Further Analysis:

In [18]:
#1 Unique values
print('The dataframe has {} boroughs and {} neighborhood.'.format(
        len(df_join['Borough'].unique()),
        df_join.shape[0]))

The dataframe has 9 boroughs and 77 neighborhood.


In [19]:
#2 Toronto coordinates
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [20]:
#3 Toronto map
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

In [28]:
for lat, lng, borough, neighborhood in zip(df_join['Latitude'], df_join['Longitude'], df_join['Borough'], df_join['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [44]:
table_can_zipinfo = soup.find('table')
colvals = table_can_zipinfo.find_all('td')

elem_cnt = len(colvals)

postcode = []
borough = []
neighborhood = []

for i in range(0, elem_cnt, 3):
    postcode.append(colvals[i].text.strip())
    borough.append(colvals[i+1].text.strip())
    neighborhood.append(colvals[i+2].text.strip())

In [45]:
df_can_postcode = pd.DataFrame(data=[postcode, borough, neighborhood]).transpose()
df_can_postcode.columns = ['Postcode', 'Borough', 'Neighborhood']

In [46]:
df_can_postcode.drop(df_can_postcode[df_can_postcode['Borough'] == 'Not assigned'].index, inplace=True)
df_can_postcode.loc[df_can_postcode.Neighborhood == 'Not assigned', "Neighborhood"] = df_can_postcode.Borough

In [47]:
df_grp_can = df_can_postcode.groupby(['Postcode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df_grp_can.columns = ['Postcode', 'Borough', 'Neighborhood']

In [48]:
df_grp_can.shape

(103, 3)